# sscws Magnetic Field Line Trace Example Jupyter Notebook

This [Jupyter notebook](https://jupyter.org/) demonstrates using the [sscws](https://pypi.org/project/sscws/) Python package to access satellite location and (modeled) magnetic field-line tracing information.  It also demonstrates how to handle an error and display the error message when an invalid request is made.   This notebook contains the following sections:

1. [Prerequisites](#Prerequisites)
2. [Setup](#Setup)
3. [Define a magnetic field line trace request](#Define-a-magnetic-field-line-trace-request)
4. [Get Locations](#Get-Locations)
5. [Error Handling](#Error-Handling)
6. [Additional Documentation](#Additional-Documentation)

## Prerequisites
Install the prerequisite software from the [Python Package Index](https://pypi.org/project/sscws/) if it is not already installed.

In [1]:
#%pip install -U sscws

## Setup
Execute some preliminary code that is necessary before the code that follows.

In [2]:
from sscws.sscws import SscWs
from sscws.bfieldmodels import BFieldModel, Tsyganenko89cBFieldModel
from sscws.coordinates import CoordinateComponent, CoordinateSystem,\
    SurfaceGeographicCoordinates
from sscws.filteroptions import LocationFilterOptions,\
    MappedRegionFilterOptions, RegionFilterOptions,\
    SpaceRegionsFilterOptions
from sscws.outputoptions import CoordinateOptions, BFieldTraceOptions,\
    DistanceFromOptions, LocationFilter, OutputOptions, RegionOptions,\
    ValueOptions
from sscws.regions import Hemisphere, HemisphereRegions
from sscws.request import DataRequest, SatelliteSpecification
from sscws.timeinterval import TimeInterval
ssc = SscWs()

## Define a magnetic field line trace request
The following code defines a request for many values including magnetic field line tracing.

In [3]:
    sats = [SatelliteSpecification('themisa', 2)]

    b_field_model = BFieldModel(external=Tsyganenko89cBFieldModel())
    coord_options = [
        CoordinateOptions(CoordinateSystem.GSE, CoordinateComponent.X),
        CoordinateOptions(CoordinateSystem.GSE, CoordinateComponent.Y),
        CoordinateOptions(CoordinateSystem.GSE, CoordinateComponent.Z),
        CoordinateOptions(CoordinateSystem.GSE, CoordinateComponent.LAT),
        CoordinateOptions(CoordinateSystem.GSE, CoordinateComponent.LON),
        CoordinateOptions(CoordinateSystem.GSE, CoordinateComponent.LOCAL_TIME)
        ]
    b_field_trace_options = [
        BFieldTraceOptions(CoordinateSystem.GEO, Hemisphere.NORTH,
                           True, True, True),
        BFieldTraceOptions(CoordinateSystem.GEO, Hemisphere.SOUTH,
                           True, True, True)
        ]

    output_options = OutputOptions(
        coord_options,
        None, None,
        RegionOptions(True, True, True, True),
        ValueOptions(True, True, True, True),
        DistanceFromOptions(True, True, True, True),
        b_field_trace_options
        )
    loc_filter = LocationFilter(0, 100000, True, True)

    hemisphere_region = HemisphereRegions(True, True)
    trace_regions = MappedRegionFilterOptions(hemisphere_region,
                                              hemisphere_region,
                                              hemisphere_region,
                                              hemisphere_region,
                                              hemisphere_region,
                                              True)
    srfo = SpaceRegionsFilterOptions(True, True, True, True, True, True,
                                     True, True, True, True, True)

    rfo = RegionFilterOptions(srfo, trace_regions, trace_regions)

    format_options = None
    
    loc_request = DataRequest('themisa b-trace locator request.',
                              TimeInterval('2020-10-02T00:00:00Z',
                                           '2020-10-02T00:10:00Z'),
                              sats, b_field_model,
                              output_options, None,
                              None, format_options)


## Get Locations
The following code submits the request created above to the SSC server and displays the results.

In [4]:
result = ssc.get_locations(loc_request)
if result['HttpStatus'] == 200:
    SscWs.print_locations_result(result)
else:
    print('ssc.get_locations failed with status = ', result['HttpStatus'])
    if 'ErrorMessage' in result:
        print('ErrorMessage = ', result['ErrorMessage'])
        print('ErrorDescription = ', result['ErrorDescription'])
    else:
        print('HttpText = ', result['ErrorText'])


themisa Gse
Time                      X                      Y                      Z                     
2020-10-02 00:00:00+00:00 25391.88098727723 76038.43619724039 17541.7997889056
2020-10-02 00:02:00+00:00 25311.055308443832 76129.3806396425 17540.543107991354
2020-10-02 00:04:00+00:00 25230.012585797875 76219.59380941694 17539.064278511694
2020-10-02 00:06:00+00:00 25148.727958863146 76308.92368175219 17537.406958795487
2020-10-02 00:08:00+00:00 25067.11881894818 76397.48618096307 17535.613675572848
2020-10-02 00:10:00+00:00 24985.290906646485 76485.30788933998 17533.59663819248
Geo North Magnetic Field-Line Trace Footpoints
Time                           Latitude         Longitude    Arc Length
2020-10-02 00:00:00+00:00             nan             nan             nan
2020-10-02 00:02:00+00:00             nan             nan             nan
2020-10-02 00:04:00+00:00             nan             nan             nan
2020-10-02 00:06:00+00:00             nan             nan         

## Error Handling
The following code creates an invalid request and demonstrates how to display the error message.

In [5]:
# add a duplicate option to cause an error
coord_options.insert(4, CoordinateOptions(CoordinateSystem.GSE, CoordinateComponent.LAT))

loc_request = DataRequest('themisa b-trace locator request.',
                          TimeInterval('2020-10-02T00:00:00Z',
                                       '2020-10-02T00:10:00Z'),
                           sats, b_field_model,
                           output_options, None,
                           None, format_options)
result = ssc.get_locations(loc_request)
if result['HttpStatus'] == 200:
    SscWs.print_locations_result(result)
else:
    print('ssc.get_locations failed with status = ', result['HttpStatus'])
    if 'ErrorMessage' in result:
        print('ErrorMessage = ', result['ErrorMessage'])
        print('ErrorDescription = ', result['ErrorDescription'])
    else:
        print('HttpText = ', result['ErrorText'])

ssc.get_locations failed with status =  400
ErrorMessage =  Invalid Request.
ErrorDescription =  4th coordinate listing option is a duplicate of an earlier option


## Additional Documentation
View the [sscws API](/WebServices/REST/py/sscws/) for additonal features.  Additional [notebook examples](https://sscweb.gsfc.nasa.gov/WebServices/REST/#Jupyter_Notebook_Examples) are also available.